## Import some python libraries

In [1]:
%matplotlib inline
import pickle
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
import random
import tensorflow.keras.utils
import tensorflow.keras.models as models
from tensorflow.keras.layers import Reshape,Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import *
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras
import joblib  # For saving and loading model
import time

## Load the data

In [2]:
Xd = pickle.load(open("data/RML2016.10a_dict.pkl",'rb'),encoding = "bytes")
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

## Split the data into training and test sets

In [3]:
# Partition the data
#  into training and test sets of the form we can train/test on 
np.random.seed(2020)
n_examples = X.shape[0]
n_train = n_examples // 2
train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]

## Convert the dependent variables (labels) into one-hoe encoding format

In [4]:
#one-hot encoding the label
lb = preprocessing.LabelBinarizer()
lb.fit(np.asarray(lbl)[:,0])
print(lb.classes_)
lbl_encoded=lb.transform(np.asarray(lbl)[:,0])
y_train=lbl_encoded[train_idx]
y_test=lbl_encoded[test_idx]

[b'8PSK' b'AM-DSB' b'AM-SSB' b'BPSK' b'CPFSK' b'GFSK' b'PAM4' b'QAM16'
 b'QAM64' b'QPSK' b'WBFM']


In [5]:
in_shp = list(X_train.shape[1:])
print(X_train.shape, in_shp)
classes = mods

(110000, 2, 128) [2, 128]


## Check the maximum and minimum values in both training and test sets

In [6]:
np.max(X_train), np.min(X_train)

(0.16046731, -0.15494554)

In [7]:
np.max(X_test), np.min(X_test)

(0.16422912, -0.15118897)

## Expand the input vector to 4D which can be fed into the CNN

In [8]:
X_train = np.expand_dims(X_train, axis = 3)
X_test = np.expand_dims(X_test, axis = 3)

In [9]:
X_test.shape[1:]

(2, 128, 1)

## Performing principal component analysis (PCA)

In [10]:
# P_components = 100

# pca = PCA(n_components = P_components, whiten = True)

# X_train_pca = pca.fit_transform(X_train)
# X_test_pca = pca.fit_transform(X_test)

## Build the CNN model

In [11]:
model = models.Sequential()
# model.add(Reshape(in_shp+[1], input_shape=in_shp))  

model.add(Conv2D(32, (1, 32), input_shape = X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (1, 32)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (1, 2)))

model.add(Conv2D(64, (2, 16)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (1, 8)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (1, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(0.2))
model.add(Dense(len(classes)))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 2, 97, 32)         1056      
_________________________________________________________________
batch_normalization (BatchNo (None, 2, 97, 32)         128       
_________________________________________________________________
activation (Activation)      (None, 2, 97, 32)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 66, 32)         32800     
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 66, 32)         128       
_________________________________________________________________
activation_1 (Activation)    (None, 2, 66, 32)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 2, 33, 32)         0

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# model.fit(X_train_pca, y_train, batch_size = 128, steps_per_epoch = 110000//128, epochs = 5, verbose = 1,
#           validation_data = (X_test_pca, y_test), validation_batch_size = 128, validation_steps = 110000//128)
start = time.time()
model.fit(X_train, y_train, batch_size = 128, steps_per_epoch = 110000//128, epochs = 5, verbose = 1,
          validation_data = (X_test, y_test), validation_batch_size = 128, validation_steps = 110000//128)
end = time.time()
training_time = end - start

print ("The training time is %.3f seconds" %(training_time))

Epoch 1/5
859/859 [==============================] - 200s 230ms/step - loss: 1.7765 - accuracy: 0.3441 - val_loss: 48.0066 - val_accuracy: 0.0917
Epoch 2/5
859/859 [==============================] - 199s 232ms/step - loss: 1.3337 - accuracy: 0.4900 - val_loss: 48.2080 - val_accuracy: 0.0917
Epoch 3/5
859/859 [==============================] - 199s 232ms/step - loss: 1.2571 - accuracy: 0.5216 - val_loss: 9.0491 - val_accuracy: 0.0918
Epoch 4/5
859/859 [==============================] - 196s 229ms/step - loss: 1.2253 - accuracy: 0.5313 - val_loss: 4.9843 - val_accuracy: 0.1112
Epoch 5/5
859/859 [==============================] - 198s 230ms/step - loss: 1.2120 - accuracy: 0.5350 - val_loss: 5.3192 - val_accuracy: 0.1411
The training time is 992.182 seconds


In [14]:
score = model.evaluate(X_test, y_test)
# score = model.evaluate(X_test_pca, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

3438/3438 [==============================] - 33s 10ms/step - loss: 5.3214 - accuracy: 0.1410
Test loss: 5.321425437927246
Test accuracy: 0.14100909233093262
